In [1]:

from torchvision.transforms import ToTensor,Resize,Compose,ToPILImage, Grayscale
from torchvision import transforms,datasets
from torch.utils.data import random_split, DataLoader, Subset,Dataset
from PIL import Image
import os
import random

n_epochs=0
batch_size=2

lr=1e-2
img_size=28

class DentalDataset(Dataset):
    def __init__(self,img_dir,transform=None,target_transform=None):
        self.img_labels=[1296,2382,2349,448,2541,1834]
        self.img_folders=['Calculus','Data caries','Gingivitis','hypodontia','Mouth Ulcer','Tooth Discoloration']
        self.img_formats=['jpg','jpeg','jpg','JPG','jpeg','jpeg']
        self.img_dir=img_dir
        self.transform=transform
        self.target_transform=target_transform

    def __len__(self):
        temo=0
        for i in self.img_labels:
            temo+=i
        return temo
    
    def __getitem__(self,idx):
        
        i=0
        x=self.img_labels[i]
        while(idx>=x):
            i+=1
            x+=self.img_labels[i]
            
        val=x-self.img_labels[i]
        val=idx-val
        strr='_ ('+str(val+1)+').'+self.img_formats[i]
        path=os.path.join(self.img_dir,self.img_folders[i],strr)
        image=Image.open(path)
        label=i
        if self.transform:
            image=self.transform(image)
        if self.target_transform:
            image=self.target_transform(image)
        return image, label

from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

current_data=DentalDataset("Adi_Dataset",transform=transform)
train_dataset, test_dataset = random_split(current_data, [0.8, 0.2])
# temp=1
# tree='({})'
# print(tree.format(temp))
# len(train_dataset)
# len(test_dataset)


In [2]:
data_root = r"Adi_Dataset"
class_folders = ['Data caries', 'Gingivitis', 'hypodontia', 'Mouth Ulcer', 'Tooth Discoloration']

# Create valid folders list
valid_folders = [os.path.join(data_root, folder) for folder in class_folders if os.path.isdir(os.path.join(data_root, folder))]

# Define the transformation for images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the dataset
dataset = datasets.ImageFolder(root=data_root, transform=transform)
# print(len(dataset))
print(dataset[1000])
# Get a smaller subset of the data (e.g., 20% of the total data)
subset_size = int(0.2 * len(dataset))  # 20% of the total data
subset_indices = random.sample(range(len(dataset)), subset_size)
subset_dataset = Subset(dataset, subset_indices)

# Split the smaller subset into training and testing sets
train_size = int(0.8 * len(subset_dataset))
test_size = len(subset_dataset) - train_size
train_dataset, test_dataset = random_split(subset_dataset, [train_size, test_size])
# Create DataLoader for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-1.8610, -1.8610, -1.8610,  ..., -2.1179, -2.1179, -2.1179],
         [-1.8610, -1.8610, -1.8610,  ..., -2.1179, -2.1179, -2.1179],
         [-1.8610, -1.8610, -1.8610,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-1.7731, -1.7731, -1.7731,  ..., -2.0357, -2.0357, -2.0357],
         [-1.7731, -1.7731, -1.7731,  ..., -2.0357, -2.0357, -2.0357],
         [-1.7731, -1.7731, -1.7731,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -

In [3]:

# train_dataloader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,pin_memory=True)
# test_dataloader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)
for i in train_dataloader:
    print(i[0].shape)
    # print(i[0][4])
    break

print(len(train_dataloader))


torch.Size([32, 3, 224, 224])
52


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.optim import lr_scheduler
from torchvision import transforms as T
from torchvision.utils import make_grid
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np


# class Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.net=nn.Sequential(nn.Linear(img_size*img_size,100),nn.ReLU(),nn.Linear(100,len(class_names)))
    
#     def forward(self,image):
#         inp=image.view(image.shape[0],-1)
#         out=self.net(inp)
#         return out

# trying resnet model

def basic(in_channels,out_channels):
    return nn.Sequential(nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1,bias=False),
                         nn.BatchNorm2d(out_channels),
                         nn.ReLU(inplace=True))

class Rs_Block(nn.Module):
    def __init__(self,in_channels):
        super().__init__()
        self.resnet=nn.Sequential(
            basic(in_channels,in_channels),
            basic(in_channels,in_channels),
        )
    
    def forward(self,x):
        return x+0.5*self.resnet(x)

class Model(nn.Module):#resnet model
    def __init__(self,in_channels=1,num_classes=6):
        super().__init__()
        self.net=nn.Sequential(
            basic(in_channels,64),nn.MaxPool2d(2),basic(64,128),nn.MaxPool2d(2),Rs_Block(128),nn.MaxPool2d(2),basic(128,256),nn.MaxPool2d(3),nn.Flatten(),nn.Linear(20736,num_classes)
        )
    
    def forward(self,x):
        return self.net(x)
model=Model().cuda() 
criterion=nn.CrossEntropyLoss(reduction='sum')
optimizer=torch.optim.SGD(model.parameters(),lr=lr)

def training():
    total_loss=0.0
    n_crct=0.0
    model.train()
    for images,labels in train_dataloader:
        targets=labels.cuda()
        y_hat=model(images.cuda())
        pred=torch.argmax(y_hat,dim=1)
        n_crct+=torch.sum(pred==targets).item()
        loss=criterion(y_hat,targets)
        total_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return total_loss/len(train_dataset),100*n_crct/len(train_dataset)

def eval():
    total_loss=0.0
    n_crct=0.0
    model.eval()
    for images, labels in test_dataloader:
        targets=labels.cuda()
        y_hat=model(images.cuda())
        pred=torch.argmax(y_hat,dim=1)
        n_crct+=torch.sum(pred==targets).item()
        loss=criterion(y_hat,targets)
        total_loss+=loss.item()
    return total_loss/len(test_dataset),100*n_crct/len(test_dataset)

def fitting(epochs):
    df=pd.DataFrame(np.empty([epochs,4]),index=np.arange(epochs),columns=['loss_train','acc_train','loss_test','acc_test'])
    progress_bar=tqdm(range(epochs))
    for i in progress_bar:
        df.iloc[i,0],df.iloc[i,1]=training()
        df.iloc[i,2],df.iloc[i,3]=eval()
        progress_bar.set_description(f"train_loss={df.iloc[i,0]}")
        progress_bar.set_postfix({"train_acc": df.iloc[i,1], "test_acc": df.iloc[i,3]})
    return df

train_history=fitting(n_epochs)
train_history

0it [00:00, ?it/s]


,loss_train,acc_train,loss_test,acc_test


In [5]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
import torchvision.models as models
import torchvision
import timm
# model = timm.create_model('efficientnet_v2_s_in21ft1k', num_classes=2)
resnet_fe=models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
for image,label in train_dataloader:
    print(image.shape)
    # x=transform(image)
    x=resnet_fe(image)
    print(x.shape)
    break
print(timm.list_pretrained("efficientvit_b2*"))

torch.Size([32, 3, 224, 224])
torch.Size([32, 1000])
['efficientvit_b2.r224_in1k', 'efficientvit_b2.r256_in1k', 'efficientvit_b2.r288_in1k']


In [7]:
# !pip install timm

In [8]:
class Expand1000toImage(nn.Module):
    def __init__(self):
        super(Expand1000toImage, self).__init__()
        
        self.fc = nn.Linear(1000, 512 * 7 * 7)  # Fully connected layer
        
        self.deconv_layers = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),  
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),    
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),     
        
        )
    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 512, 7, 7) 
        x = self.deconv_layers(x)
        return x

In [9]:
x=Expand1000toImage()(x)
vit_classifier = timm.create_model('vit_base_patch16_224_miil.in21k_ft_in1k',num_classes=5)
x=vit_classifier(x)
print(x.shape)

torch.Size([32, 5])


In [10]:
class Hybrid_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet_fe=models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
        self.converter=Expand1000toImage()
        self.vit_classifier = timm.create_model('vit_base_patch16_224_miil.in21k_ft_in1k', num_classes=6)

    def forward(self,x):
        x=self.resnet_fe(x) 
        x=self.converter(x)
        x=self.vit_classifier(x)
        return x

In [13]:
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model=Hybrid_model()
# model_state=torch.load('best_hybrid_model.pth')
# model.load_state_dict(model_state)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

num_epochs = 35
best_val_accuracy = 0
patience = 5
patience_counter = 0

print('entered training')
print(len(train_dataloader))
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    i=1
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if(i%8==0):
            print(i)
        
        i+=1

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader):.4f}, Training Accuracy: {train_accuracy:.2f}%")

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

    # Save best model with early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_hybrid_model.pth')
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping due to no improvement.")
            break
    scheduler.step()

print(f"Best Validation Accuracy: {best_val_accuracy:.2f}%")


entered training
52


KeyboardInterrupt: 